# Is duckDB ammenible to our data structure? Should we build a class around it?

***

In [1]:
import duckdb as ddb
import pandas as pd
import numpy as np

## 1. How quick can we query protein files?

In [2]:
# in memory db - lost when process ends
conn = ddb.connect()

In [3]:
# run query on set of protein files
df = conn.execute("""
    SELECT *
    FROM read_csv_auto('../data/taxa/proteins/taxa*.csv', header=True)
    LIMIT 10
""").df()

In [14]:
df

,seq_id,protein_seq,protein_desc,protein_len
0,4417.0,MSKYDEALARMQLRSNEAFERITGQTVEQYQQEQRQNRMPTTAQLA...,hypothetical protein,4602
1,4417.1,MTEEETLGILSENPEASPMKINPKHLEKFFKDNHTPNTKISISIAN...,hypothetical protein,53
2,4417.2,MTQHLSHLSTTPNTQKPITLYISGLTHDGRGIATYDDTHGDKSGKK...,23S rRNA (uracil(1939)-C(5))-methyltransferase...,476
3,4417.3,MSNNIQHLRNIAIIAHVDHGKTTLVDKLLQQSGALGERAGEIERVM...,translational GTPase TypA,615
4,4417.4,MKLNKIALALIATATTAATAGVTVTPLIGYQHQNEAHKKQREIFHT...,OmpA family protein,423
5,4417.5,MRAPRLGVNIDHIATLRNVRGVDYPCPIKGALVCQRAGADGITLHL...,pyridoxine 5'-phosphate synthase,244
6,4417.6,MSLQSLKIQSLPRFSLNFAANIVAALWMLVGSVRAFNWVKPTFGQF...,caspase family protein,479
7,4417.7,MTSIYLLIPLSLMLFVVAIWAIAYAVKSNQFEDLDNAPDQIILDDR...,cbb3-type cytochrome oxidase assembly protein ...,70
8,4417.8,MSIVLLAPALAMGFLGSPHCMGMCGGIVTAFGISMKNLSPQKRGML...,sulfite exporter TauE/SafE family protein,280
9,4417.9,MLEIRHLQMLSILARHGSLVMTADELNLTASAISHQLKELESYYDI...,LysR family transcriptional regulator,295


## 2. Make it a virtual table

In [15]:
# all protein files
df = conn.execute("""
    SELECT *
    FROM read_csv_auto('../data/taxa/proteins/taxa*.csv', header=True)
""").df()
conn.register('proteins', df)

its too big to be a virtual table...

## 3. Make a database of file and add an actual table

In [4]:
conn = ddb.connect(database="./dbs/test_db")

In [7]:
conn.execute("""
    CREATE OR REPLACE TABLE proteins AS
        SELECT
            "seq_id"::STRING AS seq_id.m
            "protein_seq"::STRING AS protein_seq,
            "protein_desc"::STRING AS protein_desc,
            "protein_len"::INT AS protein_len
        FROM read_csv_auto('../data/taxa/proteins/taxa*.csv', header=True)
""")

In [8]:
conn.execute("FROM proteins LIMIT 10").df()

,seq_id,protein_seq,protein_desc,protein_len
0,4417.0,MSKYDEALARMQLRSNEAFERITGQTVEQYQQEQRQNRMPTTAQLA...,hypothetical protein,4602
1,4417.1,MTEEETLGILSENPEASPMKINPKHLEKFFKDNHTPNTKISISIAN...,hypothetical protein,53
2,4417.2,MTQHLSHLSTTPNTQKPITLYISGLTHDGRGIATYDDTHGDKSGKK...,23S rRNA (uracil(1939)-C(5))-methyltransferase...,476
3,4417.3,MSNNIQHLRNIAIIAHVDHGKTTLVDKLLQQSGALGERAGEIERVM...,translational GTPase TypA,615
4,4417.4,MKLNKIALALIATATTAATAGVTVTPLIGYQHQNEAHKKQREIFHT...,OmpA family protein,423
5,4417.5,MRAPRLGVNIDHIATLRNVRGVDYPCPIKGALVCQRAGADGITLHL...,pyridoxine 5'-phosphate synthase,244
6,4417.6,MSLQSLKIQSLPRFSLNFAANIVAALWMLVGSVRAFNWVKPTFGQF...,caspase family protein,479
7,4417.7,MTSIYLLIPLSLMLFVVAIWAIAYAVKSNQFEDLDNAPDQIILDDR...,cbb3-type cytochrome oxidase assembly protein ...,70
8,4417.8,MSIVLLAPALAMGFLGSPHCMGMCGGIVTAFGISMKNLSPQKRGML...,sulfite exporter TauE/SafE family protein,280
9,4417.9,MLEIRHLQMLSILARHGSLVMTADELNLTASAISHQLKELESYYDI...,LysR family transcriptional regulator,295


In [9]:
conn.execute("FROM proteins SELECT * WHERE protein_len<50").df()

,seq_id,protein_seq,protein_desc,protein_len
0,4417.94,MFVSEWVYNLLVVMGGGEVWGFGGLGFDKMTWCIKAYVGRASLPS,hypothetical protein,45
1,4417.114,MPLHDNIRKFREQKQWSQEYMAEQLGLSKNGYAKIERGESRPSLDRLE,helix-turn-helix transcriptional regulator,48
2,4417.116,MQVLSSLKSAKNRHEDCQVVRRRGRTFVICKSNPRFKAVQGGKKRK,type B 50S ribosomal protein L36,46
3,4417.121,MSKFLVSTRINRYEKGVYQPNFGILEKMAMALDVPVIYFLAMMNWHK,helix-turn-helix transcriptional regulator,47
4,4417.185,MKRTFQPSVLKRKRTHGFRARMATKNGRQVLARRRAKGRHRLTV,50S ribosomal protein L34,44
...,...,...,...,...
405400,3361.3628,MKVRASVKKLCRNCKIVRREGVVRVICSAEPRHKQRQG,50S ribosomal protein L36,38
405401,3361.3861,MNAQSKIEQHSPIRTDGFEIVEYRASTTAGIAGSLPYLAYRVLGA,hypothetical protein,45
405402,3361.3879,MWYFAWILGVLLACSFGIVNALWLETTQDLDEGEAGDD,cytochrome bd-I oxidase subunit CydX,38
405403,3361.3889,MACSIDRSYIGRIERGEVNITVEKLYRIASLLSCDPASLLPLVSELQG,helix-turn-helix domain-containing protein,48


## 4. create table with primary keys

Summary: it sucks. YOu cannot modify a column to be foreign key, and you cant create the scheme with foreign key and then import data from CSV unless the headers match percetly, so have to parse, rename, save, create scheme then parse again

In [30]:
conn.execute("""
    CREATE OR REPLACE TABLE taxa AS
        SELECT
            "taxid"::INT AS ncbi_taxid,
            "record_name"::STRING AS record_name,
            "filepath"::STRING AS filepath,
            "taxonomy"::STRING AS taxonomy,
            "organism"::STRING AS organism,
            "bacdive_id"::INT AS bacdive_id,
            "ogt_raw":: STRING AS ogt_scraped_string
        FROM read_csv_auto('../data/taxa/taxa_info_and_ogt.csv', header=True)
""")

In [31]:
conn.execute("DESCRIBE taxa").df()

,column_name,column_type,null,key,default,extra
0,ncbi_taxid,INTEGER,YES,NaN,NaN,NaN
1,record_name,VARCHAR,YES,NaN,NaN,NaN
2,filepath,VARCHAR,YES,NaN,NaN,NaN
3,taxonomy,VARCHAR,YES,NaN,NaN,NaN
4,organism,VARCHAR,YES,NaN,NaN,NaN
5,bacdive_id,INTEGER,YES,NaN,NaN,NaN
6,ogt_scraped_string,VARCHAR,YES,NaN,NaN,NaN


In [2]:
conn = ddb.connect(database="./dbs/test_db")

In [3]:

conn.execute("""
CREATE SEQUENCE seq_taxa_index START 1
""")
conn.execute("""
CREATE OR REPLACE TABLE taxa(
    taxa_index INT PRIMARY KEY default nextval('seq_taxa_index')-1,
    ncbi_taxid INT
)
""")


In [4]:
conn.execute("INSERT INTO taxa(ncbi_taxid) VALUES (1)")

In [5]:
conn.execute("SELECT * FROM taxa").df(
)

,taxa_index,ncbi_taxid
0,0,1


Try creating a virtual table with the correct names, THEN using the copy on the actual table with the correct scheme

In [2]:
conn = ddb.connect(database="./dbs/test_db")

In [14]:
conn.execute("""
CREATE OR REPLACE TABLE taxa(
    taxa_index INT PRIMARY KEY NOT NULL,
    ncbi_taxid INT NOT NULL,
    record_name STRING,
    filepath STRING,
    taxonomy STRING,
    organism STRING,
    bacdive_id INT,
    ogt_scraped_string STRING
)
""")

In [7]:
conn.execute("""CREATE OR REPLACE TEMP TABLE taxa_tmp AS 
        SELECT
            "column0"::INT AS taxa_index,
            "taxid"::INT AS ncbi_taxid,
            "record_name"::STRING AS record_name,
            "filepath"::STRING AS filepath,
            "taxonomy"::STRING AS taxonomy,
            "organism"::STRING AS organism,
            "bacdive_id"::INT AS bacdive_id,
            "ogt_raw":: STRING AS ogt_scraped_string
        FROM read_csv_auto('../data/taxa/taxa_info_and_ogt.csv', header=True)""")
conn.execute("COPY taxa_tmp TO 'taxa_tmp.csv' (HEADER)")

In [15]:
conn.execute("COPY taxa FROM 'taxa_tmp.csv' ( HEADER )")

In [17]:
conn.execute("DESCRIBE taxa").df()

,column_name,column_type,null,key,default,extra
0,taxa_index,INTEGER,NO,NaN,NaN,NaN
1,ncbi_taxid,INTEGER,NO,NaN,NaN,NaN
2,record_name,VARCHAR,YES,NaN,NaN,NaN
3,filepath,VARCHAR,YES,NaN,NaN,NaN
4,taxonomy,VARCHAR,YES,NaN,NaN,NaN
5,organism,VARCHAR,YES,NaN,NaN,NaN
6,bacdive_id,INTEGER,YES,NaN,NaN,NaN
7,ogt_scraped_string,VARCHAR,YES,NaN,NaN,NaN


In [25]:
conn.execute("CREATE INDEX pkey ON taxa (taxa_index)")

In [26]:
conn.execute("SELECT * FROM duckdb_indexes()").df()

,schema_name,schema_oid,index_name,index_oid,table_name,table_oid,is_unique,is_primary,expressions,sql
0,main,1,pkey,1334,taxa,1294,False,False,NaN,CREATE INDEX pkey ON taxa (taxa_index);


In [27]:
conn.execute("SELECT * FROM duckdb_constraints()").df()

,schema_name,schema_oid,table_name,table_oid,constraint_index,constraint_type,constraint_text,expression,constraint_column_indexes,constraint_column_names
0,main,1,taxa,1294,0,PRIMARY KEY,PRIMARY KEY(taxa_index),NaN,[0],[taxa_index]
1,main,1,taxa,1294,1,NOT NULL,NOT NULL,NaN,[0],[taxa_index]
2,main,1,taxa,1294,2,NOT NULL,NOT NULL,NaN,[1],[ncbi_taxid]


## 5. Do a foreign key assignment

In [30]:
conn.execute("""
CREATE OR REPLACE TABLE taxa(
    taxa_index INT PRIMARY KEY NOT NULL,
    ncbi_taxid INT NOT NULL,
    record_name STRING,
    filepath STRING,
    taxonomy STRING,
    organism STRING,
    bacdive_id INT,
    ogt_scraped_string STRING
)
""")
conn.execute("""
CREATE OR REPLACE TABLE taxa_16s(
    taxa_index INT PRIMARY KEY NOT NULL,
    seq_16srRNA STRING,
    FOREIGN KEY (taxa_index) REFERENCES taxa(taxa_index) 
)""")

Fill in the taxa table

In [31]:
conn.execute("""CREATE OR REPLACE TEMP TABLE taxa_tmp AS 
        SELECT
            "column0"::INT AS taxa_index,
            "taxid"::INT AS ncbi_taxid,
            "record_name"::STRING AS record_name,
            "filepath"::STRING AS filepath,
            "taxonomy"::STRING AS taxonomy,
            "organism"::STRING AS organism,
            "bacdive_id"::INT AS bacdive_id,
            "ogt_raw":: STRING AS ogt_scraped_string
        FROM read_csv_auto('../data/taxa/taxa_info_and_ogt.csv', header=True)""")
conn.execute("COPY taxa_tmp TO 'taxa_tmp.csv' (HEADER)")
conn.execute("COPY taxa FROM 'taxa_tmp.csv' ( HEADER )")

now fill in the 16s table

In [33]:
conn.execute("COPY taxa_16s FROM '../data/taxa/16s_rRNA.csv' ( HEADER )")

Try to execute an inner join

In [49]:
df = conn.execute("""
    SELECT (taxa_16s.seq_16srRNA, taxa.ncbi_taxid)
        FROM taxa_16s INNER JOIN taxa
        ON taxa_16s.taxa_index = taxa.taxa_index
    WHERE taxa.ncbi_taxid<100000
""").df()

In [53]:
df = df['main.row(taxa_16s."seq_16srRNA", taxa.ncbi_taxid)'].apply(pd.Series)

In [57]:
df.isna().sum()

seq_16srRNA    0
ncbi_taxid     0
dtype: int64

In [67]:
df = conn.execute("""
    SELECT (taxa_16s.seq_16srRNA)
        FROM taxa_16s INNER JOIN taxa
        ON taxa_16s.taxa_index = taxa.taxa_index
    WHERE taxa.ncbi_taxid<1000000
""").df()

In [68]:
(df['seq_16srRNA'].apply(len) == 0).sum()

0

where are the taxa without sequances?

In [77]:
conn.execute("""
    SELECT * FROM taxa_16s
    LIMIT 10 
""").df()

,taxa_index,seq_16srRNA
0,0,ACGATGGAGAGTTTGATCCTGGCTCAGGATGAACGCTAGCGGCAGG...
1,2,CAACTTGAGAGTTTGATCCTGGCTCAGAGCGAACGCTGGCGGCAGG...
2,1,TCAACGGAGAGTTTGATCCTGGCTCAGGACGAACGCTGGCGGCGTG...
3,4,CAAATGGAGAGTTTGATCCTGGCTCAGGATGAACGCTGGCGGCATG...
4,11,GAACCTGAGAGTTTGATCCTGGCTCAGAGCGAACGCTGGCGGCATG...
5,8,None
6,10,CAACTAAAGAGTTTGATCCTGGCTCAGATTGAACGCTGGCGGCATG...
7,7,ACAATGGAGAGTTTGATCCTGGCTCAGGATGAACGCTAGCGGGAGG...
8,5,TTGTTGGAGAGTTTGATCCTGGCTCAGGACGAACGCTGGCGGCGTG...
9,6,GAACTGAAGAGTTTGATCCTGGCTCAGATTGAACGCTGGCGGCATG...


they are there, but dissappeared when we inner joined?

In [79]:
conn.execute("""
    SELECT (taxa_16s.seq_16srRNA)
        FROM taxa_16s INNER JOIN taxa
        ON taxa_16s.taxa_index = taxa.taxa_index
    WHERE taxa_16s.taxa_index=8
""").df()

,seq_16srRNA
0,None


In [85]:
df = conn.execute("""
    SELECT taxa.taxa_index, taxa_16s.seq_16srRNA
        FROM taxa_16s INNER JOIN taxa
        ON taxa_16s.taxa_index = taxa.taxa_index
""").df()

In [86]:
df.head(10)

,taxa_index,seq_16srRNA
0,0,ACGATGGAGAGTTTGATCCTGGCTCAGGATGAACGCTAGCGGCAGG...
1,2,CAACTTGAGAGTTTGATCCTGGCTCAGAGCGAACGCTGGCGGCAGG...
2,1,TCAACGGAGAGTTTGATCCTGGCTCAGGACGAACGCTGGCGGCGTG...
3,4,CAAATGGAGAGTTTGATCCTGGCTCAGGATGAACGCTGGCGGCATG...
4,11,GAACCTGAGAGTTTGATCCTGGCTCAGAGCGAACGCTGGCGGCATG...
5,8,None
6,10,CAACTAAAGAGTTTGATCCTGGCTCAGATTGAACGCTGGCGGCATG...
7,7,ACAATGGAGAGTTTGATCCTGGCTCAGGATGAACGCTAGCGGGAGG...
8,5,TTGTTGGAGAGTTTGATCCTGGCTCAGGACGAACGCTGGCGGCGTG...
9,6,GAACTGAAGAGTTTGATCCTGGCTCAGATTGAACGCTGGCGGCATG...


All looks good...